In [6]:
import xml.etree.ElementTree as ET

def parse_tweet(tweet):
    text = ""
    labels = []
    
    # Recorre todos los elementos y texto dentro del tweet
    for element in tweet.iter():
        if element.tag == "sentiment":
            # Extrae el aspecto y la polaridad
            aspect = element.attrib["aspect"]
            polarity = element.attrib["polarity"]
            
            # Divide el texto en palabras y asigna las etiquetas BIO
            words = element.text.split()
            labels.extend([f"B-{aspect}"] + [f"I-{aspect}"] * (len(words) - 1))
            text += element.text + " "
        else:
            # Captura el texto que está fuera de los elementos <sentiment>
            if element.text:
                words = element.text.split()
                labels.extend(["O"] * len(words))
                text += element.text + " "
        
        # Captura el texto de cola (tail) que está después de un elemento <sentiment>
        if element.tail and element.tail.strip():
            words = element.tail.split()
            labels.extend(["O"] * len(words))
            text += element.tail.strip() + " "

    return text.strip(), labels

In [7]:
tweet_xml = '''<tweet id="456544889786728451">
<sentiment aspect="Afición" polarity="P">Los que esta noche van a la redonda a celebrar la victoria</sentiment>
del
<sentiment aspect="Equipo-Real_Madrid" polarity="P">Real Madrid</sentiment>
les espero para cuando suba el
<sentiment aspect="Equipo" polarity="P">@realmurciacfsad</sentiment>
a primera.
</tweet>'''

root = ET.fromstring(tweet_xml)
text, labels = parse_tweet(root)
print({"text": text, "labels": labels})

{'text': 'Los que esta noche van a la redonda a celebrar la victoria del Real Madrid les espero para cuando suba el @realmurciacfsad a primera.', 'labels': ['B-Afición', 'I-Afición', 'I-Afición', 'I-Afición', 'I-Afición', 'I-Afición', 'I-Afición', 'I-Afición', 'I-Afición', 'I-Afición', 'I-Afición', 'I-Afición', 'O', 'B-Equipo-Real_Madrid', 'I-Equipo-Real_Madrid', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Equipo', 'O', 'O']}


In [11]:
import csv
def process_xml_to_csv(xml_file, csv_file):
    # Parsear el archivo XML
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Abrir el archivo CSV para escribir
    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file, delimiter=";")
        # Escribir la cabecera del CSV
        writer.writerow(["text", "labels"])
        
        # Recorrer cada tweet en el XML
        for tweet in root.findall("tweet"):
            text, labels = parse_tweet(tweet)
            # Escribir una fila en el CSV
            writer.writerow([text, " ".join(labels)])  # Convertir la lista de etiquetas a una cadena

# Ejemplo de uso
xml_file = "../Data/socialtv-train-tagged.xml"  # Ruta al archivo XML de entrada
csv_file = "tweets_preprocessed.csv"  # Ruta al archivo CSV de salida
process_xml_to_csv(xml_file, csv_file)

print(f"Procesamiento completado. Los resultados se han guardado en {csv_file}.")


Procesamiento completado. Los resultados se han guardado en tweets_preprocessed.csv.
